In [ ]:
# /**********************************************************************************************************

# File Name		: 	streaming_tweet_bq.ipynb
# Purpose			:   Loading streaming tweets into bigquery table
# Author			:   DeepSphere.AI, Inc.
# Date and Time 	: 	03/16/2021 10:30 hrs
# Version			: 	1.0 

# /************************************************************************************************************

from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
import json

vAR_access_token = '1124283270-zjlHsFC9EkqvC1PAybE6NE4m43OPX8dZdM9GmnS'
vAR_access_token_secret =  "asJLhzYTI8dITcsqlSS4sgVE3ihN6b4TGVUoKHSTtAVJ3"
vAR_consumer_key =  "S8PpgQWTXiWcmkdd5CTuUMYzT"
vAR_consumer_secret =  "d85fJZ0t527YogNae17MqVIh452Zo8ZmQrnTPuz76j5p5Ps9Sj"

def upload_bq(rows_to_insert):
    from google.cloud import bigquery

    # Construct a BigQuery client object.
    client = bigquery.Client()

    # TODO(developer): Set table_id to the ID of table to append to.
    table_id = "vast-verve-292018.ds_ai.ds_ai_stream_tweets"

    errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
    if errors == []:
        print("New rows have been added.")
    else:
        print("Encountered errors while inserting rows: {}".format(errors))

def get_tweet_count():
    from google.cloud import bigquery
    import json
    # Construct a BigQuery client object.
    vAR_client = bigquery.Client()

    # TODO(developer): Set table_id to the ID of the destination table.
    vAR_sql = """
        SELECT count(1) as count FROM `vast-verve-292018.ds_ai.ds_ai_stream_tweets`
    """

    # Start the query, passing in the extra configuration.
    query_job = vAR_client.query(vAR_sql)  # Make an API request.
    results =query_job.result()  # Wait for the job to complete.
    for res in results:
        count = res.count
    return count

class twitterAuth():

    def authenticateTwitterApp(self):
        vAR_auth = OAuthHandler(vAR_consumer_key, vAR_consumer_secret)
        vAR_auth.set_access_token(vAR_access_token, vAR_access_token_secret)
        return vAR_auth
class TwitterStreamer():

    def __init__(self):
        self.twitterAuth = twitterAuth()
        

    def stream_tweets(self):
        while True:
            vAR_listener = ListenerTS() 
            vAR_auth = self.twitterAuth.authenticateTwitterApp()
            rName_stream = Stream(vAR_auth, vAR_listener)
            rName_stream.filter(track=["covid"], stall_warnings=True, languages= ["en"])

class ListenerTS(StreamListener):

    def on_data(self, raw_data):
            tweet_count = get_tweet_count()
            if tweet_count>100:
                raise Exception("We got more than 100+ tweets")
            vAR_dtime = datetime.now()
            print(f'[{vAR_dtime.hour}:{vAR_dtime.minute}:{vAR_dtime.second}] sending tweet')
            upload_bq([{"tweet":raw_data}])

if __name__ == "__main__":
    try:
        # Loading Streaming tweets into Bigquery Table
        rName_TS = TwitterStreamer()
        rName_TS.stream_tweets()
    except:
        !gsutil rm -r gs://videoai_intelligence/VideoAI/youtube_videos3
        !find /home/dsailabusr1/PRAKASH/youtube_videos3 -type f -delete
        print('Already processed youtube files deleted automatically')
        # Twitter Data Load into Bigquery Table
        %run /home/dsailabusr1/PRAKASH/DE-Framework/DE-Framework-Impl/Framework/twitter_load.ipynb
        vAR_twitter_load_flag = twitter_load()
        print('twitter_load flag - ',vAR_twitter_load_flag)
        
        # Youtube data load into Bigquery Table
        %run /home/dsailabusr1/PRAKASH/DE-Framework/DE-Framework-Impl/Framework/youtube_load.ipynb
        vAR_youtube_load_flag = youtube_load()
        print('youtube_load flag - ',vAR_youtube_load_flag)
        
        # Image data load into Bigquery Table
        %run /home/dsailabusr1/PRAKASH/DE-Framework/DE-Framework-Impl/Framework/image_load.ipynb
        vAR_image_load_flag = image_load()
        print('image_load flag - ',vAR_image_load_flag)
        
        # Integrating all above table into single Bigquery Table
        %run /home/dsailabusr1/PRAKASH/DE-Framework/DE-Framework-Impl/Framework/data_integration.ipynb
        vAR_integrate_load_flag = data_integrate()
        print('integrated_data_load flag - ',vAR_integrate_load_flag)
        
        # Applying Kmeans clustering Model to above integrated data
        %run /home/dsailabusr1/PRAKASH/DE-Framework/DE-Framework-Impl/Framework/bigqueryML.ipynb
        vAR_model_flag = kmeans_model()
        print('model_prediction flag - ',vAR_model_flag)
        
        # Customizing model output
        %run /home/dsailabusr1/PRAKASH/DE-Framework/DE-Framework-Impl/Framework/customer_group.ipynb
        vAR_model_op_flag = bq_insert_bqml()
        print('model_op_customized flag - ',vAR_model_op_flag)
        
# /**********************************************************************************************************
# Disclaimer.

# We are providing this code block strictly for learning and researching, this is not a production ready code.
# We have no liability on this particular code under any circumstances; users should
# use this code on their own risk. All software,hardware and other products that are
# referenced in these materials belong to the respective vendor who developed or who owns 
# this product.
# /**********************************************************************************************************
